In [ ]:
#Script that performs and saves the land movement IDW interpolation (25m grid)

import geopandas as gpd
import numpy as np
import rasterio
from rasterio.transform import from_origin
from scipy.spatial import KDTree
from pyproj import CRS

# Load the point and boundary data
point_data = gpd.read_file('D:\\FOLDER FROM THESIS\\THESIS\\Data\\Land movement\\Vertikal SDFE 80m gitter DK 2020_25832.shp')
boundary = gpd.read_file('D:\\FOLDER FROM THESIS\\THESIS\\Data\\VektorDanmark\\Danmark.shp')

# Validate CRS
assert point_data.crs == 'EPSG:25832'
assert boundary.crs == 'EPSG:25832'

# Extract coordinates and values
coordinates = point_data.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
values = point_data['vel_vert'].values

# KDTree for spatial search
tree = KDTree(coordinates)

# Define grid based on boundary
minx, miny, maxx, maxy = boundary.geometry.total_bounds
grid_res = 25  # Grid resolution
grid_x, grid_y = np.mgrid[minx:maxx:grid_res, miny:maxy:grid_res]

# IDW Interpolation
n_neighbors = 3
distances, indices = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=n_neighbors)
weights = 1.0 / np.maximum(distances, 1e-10)**2  # Avoid division by zero
interpolated_values = np.sum(weights * values[indices], axis=1) / np.sum(weights, axis=1)

#reshaoe to match grid x and y
prediction = interpolated_values.reshape(grid_x.shape)

# Rotate the prediction array 90 degrees counterclockwise
rotated_prediction = np.rot90(prediction, k=1)

# Save as rotated raster
output_path = 'D:\\FOLDER FROM THESIS\\THESIS\\Data\\Land movement\\Interpolations\\IDW_interpolation_25m.tif'
with rasterio.open(output_path, 'w', driver='GTiff',
                   height=rotated_prediction.shape[0], width=rotated_prediction.shape[1], count=1,
                   dtype=rotated_prediction.dtype, crs=CRS.from_epsg(25832).to_wkt(),
                   transform=from_origin(minx, maxy, grid_res, grid_res)) as dst:
    dst.write(rotated_prediction[np.newaxis, ...])